In [ ]:
# Importing the libary
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import glob



In [2]:
# Importing path from src
import sys
import yaml

sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))

from src.config import RAW_DATA_DIR, PROCESSED_DATA_DIR, BASE_DIR
from src.log_handler import LOGGER

with open(os.path.join(BASE_DIR, "config/config.yaml"), "r") as file:
    SETTINGS = yaml.safe_load(file)
LOGGER.info("Succesfully imported the file")

[2025-09-06 00:39:14,823] 35 root - INFO - Logging initialized successfully
[2025-09-06 00:39:14,826] 12 root - INFO - Succesfully imported the file


In [3]:
# Loading the dotenv file
load_dotenv()

True

In [4]:
# Laading the api key fron environment variable
API_KEY = os.getenv("TWELVE_DATA_API_KEY")

In [5]:
day = SETTINGS["data"]["days"]
print(day)

60


In [6]:
# Datetime variiable
day = SETTINGS["data"]["days"]
interval_time = SETTINGS['data']['interval']
output_size = SETTINGS['data']['output_size']
symb = SETTINGS['data']['n_symbol']

end_date= pd.to_datetime(datetime.now().strftime("%Y-%m-%d"))
start_date = end_date- timedelta(days=day)

td = TDClient(API_KEY)

# Fetching data
obj = td.time_series(
    symbol = symb,
    interval=interval_time,
    start_date = start_date,
    end_date= end_date,
    outputsize = output_size
)

df = obj.as_pandas()

In [7]:
df.sort_index(ascending=True)
df.tail()

,open,high,low,close,volume
datetime,,,,,
2025-08-18 14:48:00,1246.51501,1246.51501,1246.19495,1246.35535,2085
2025-08-18 14:47:00,1246.52002,1246.96997,1246.00745,1246.00745,2640
2025-08-18 14:46:00,1245.72998,1245.79504,1245.72998,1245.79504,1875
2025-08-18 14:45:00,1246.06995,1246.08997,1245.92004,1245.92505,3204
2025-08-18 14:44:00,1246.19995,1246.19995,1246.15503,1246.15503,2105


In [8]:
# Saving the loaded data to local 

df.to_csv(os.path.join(RAW_DATA_DIR, f"{datetime.now().strftime('%Y-%m-%d')}_raw_data.csv"), mode='a')

keep_file = SETTINGS['data']['keep_days']
for file in glob.glob(os.path.join(RAW_DATA_DIR, "*.csv")):
    file_time= datetime.fromtimestamp(os.path.getmtime(file))
    if (datetime.now() - file_time).days >keep_file:
        os.remove(file)
        print(f"Deleted old file: {file}")